#Setup

In [13]:
# Install necessary packages
%pip install ultralytics
%pip install tensorflow
%pip install opencv-python-headless
%pip install pandas
%pip install matplotlib
!pip install ultralytics
!yolo model=yolov8n.pt

# Import necessary libraries
import cv2
import os # Import the os module
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import tensorflow as tf


WARNING ⚠️ 'mode' argument is missing. Valid modes are frozenset({'train', 'export', 'val', 'benchmark', 'predict', 'track'}). Using default 'mode=train'.
100% 6.25M/6.25M [00:00<00:00, 73.3MB/s]
WARNING ⚠️ 'data' argument is missing. Using default 'data=coco8.yaml'.
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stri

In [3]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Frame Extraction
Extract frames from the input video at a configurable interval (e.g., one frame per second).

In [5]:
#import cv2
#import os

def extract_frames(video_path, output_folder, interval_sec=1):
    """Extract frames from a video file at the given interval in seconds."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * interval_sec)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Save frame if at the interval
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1
        frame_count += 1

    cap.release()
    print(f"Extracted {saved_count} frames from {video_path}")

# Example usage:
# extract_frames('data/bird_timelapse.mp4', 'output/frames', interval_sec=1)


# 2. Bird Detection
Use a pre-trained object detection model (e.g., YOLOv8) to locate birds in each frame. In this example, we use the Ultralytics YOLO implementation.

python
Copy


In [6]:
#from ultralytics import YOLO
#import cv2

def detect_birds_in_frame(frame_path, model, conf_threshold=0.5):
    """
    Detect birds in a frame using YOLO.
    Returns list of bounding boxes with the format: [x1, y1, x2, y2]
    """
    frame = cv2.imread(frame_path)
    results = model(frame)
    detections = []

    # Loop through detected objects in the first result
    for result in results:
        # Assume that the model is trained/detected for birds and filter by confidence.
        if result.conf[0] >= conf_threshold:
            x1, y1, x2, y2 = map(int, result.xyxy[0])
            detections.append((x1, y1, x2, y2))
    return detections

# Load YOLOv8 model (customize model path if needed)
# model = YOLO("yolov8_birds.pt")
# Example usage:
# boxes = detect_birds_in_frame('output/frames/frame_00001.jpg', model)


# 3. Species Classification
Crop the detected regions and classify each bird using a deep learning classification model (TensorFlow or PyTorch). Below is an example using TensorFlow/Keras:

In [7]:
#import cv2
#import numpy as np
#import tensorflow as tf

# Assume you have a mapping from model output indices to species names
SPECIES_LABELS = {0: 'Sparrow', 1: 'Robin', 2: 'Blue Jay'}

def preprocess_crop(crop, target_size=(224, 224)):
    """Resize and normalize the image crop."""
    crop_resized = cv2.resize(crop, target_size)
    crop_normalized = crop_resized.astype("float32") / 255.0
    # Expand dimensions to match model input: (1, height, width, channels)
    return np.expand_dims(crop_normalized, axis=0)

def classify_bird(cropped_img, model):
    """
    Classify the bird in the cropped image.
    Returns the species name.
    """
    preprocessed_img = preprocess_crop(cropped_img)
    predictions = model.predict(preprocessed_img)
    species_index = np.argmax(predictions, axis=1)[0]
    species_name = SPECIES_LABELS.get(species_index, "Unknown")
    return species_name

# Example usage:
# classification_model = tf.keras.models.load_model('models/bird_classifier.h5')
# crop = frame[y1:y2, x1:x2]
# species = classify_bird(crop, classification_model)


# 4. Counting & Data Storage
Keep track of detected and classified birds, avoid duplicate counts across frames (if necessary by implementing a simple tracking mechanism), and store results in a structured format.

In [8]:
#import pandas as pd
#from datetime import timedelta

def update_species_counts(counts_dict, timestamp, species):
    """
    Update the count of species at the given timestamp.
    counts_dict should be a list of dicts.
    """
    counts_dict.append({
        'timestamp': timestamp,
        'species': species,
        'count': 1  # For each detected instance; you can later aggregate per frame
    })

def save_counts_to_csv(data, output_csv):
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"Species counts saved to {output_csv}")

# Example usage:
# species_counts = []
# update_species_counts(species_counts, "00:00:05", "Sparrow")
# save_counts_to_csv(species_counts, "output/species_counts.csv")


# 5. Visualization
Generate a time-series plot showing species counts over time using Matplotlib or Plotly.

In [9]:
#import matplotlib.pyplot as plt
#import pandas as pd

def plot_species_counts(csv_file):
    df = pd.read_csv(csv_file)
    # Convert timestamp to a datetime object if necessary
    # For this example, assume the 'timestamp' column is in seconds.
    df['timestamp'] = pd.to_timedelta(df['timestamp'].astype(float), unit='s')

    # Pivot the dataframe to have timestamps as index and species as columns
    pivot_df = df.pivot_table(index='timestamp', columns='species', values='count', aggfunc='sum').fillna(0)

    pivot_df.plot(figsize=(10, 6))
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.title('Bird Species Count Over Time')
    plt.legend(title='Species')
    plt.tight_layout()
    plt.show()

# Example usage:
# plot_species_counts("output/species_counts.csv")


# 6. Orchestrating the Pipeline

In [12]:
def main():
    video_path = '/content/drive/MyDrive/Dev/BirdWatcher/Input/output.mp4'
    frames_folder = 'Output/frames'
    counts_csv = 'Output/species_counts.csv'
    extraction_interval = 1  # in seconds

    # Step 1: Extract frames from video
    extract_frames(video_path, frames_folder, extraction_interval)

    # Load your models
    detection_model = YOLO("yolov8_birds.pt")
    classification_model = tf.keras.models.load_model('models/bird_classifier.h5')

    species_counts = []

    # Process each extracted frame
    import glob
    frame_files = sorted(glob.glob(f"{frames_folder}/*.jpg"))

    for i, frame_file in enumerate(frame_files):
        timestamp = i * extraction_interval  # seconds elapsed

        # Step 2: Detect birds in the frame
        boxes = detect_birds_in_frame(frame_file, detection_model)
        frame = cv2.imread(frame_file)

        # Process each detected bird
        for box in boxes:
            x1, y1, x2, y2 = box
            cropped_bird = frame[y1:y2, x1:x2]

            # Step 3: Classify the bird species
            species = classify_bird(cropped_bird, classification_model)

            # Step 4: Update count data
            update_species_counts(species_counts, timestamp, species)

            # Optional: Draw bounding box and species label on the frame for visualization
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, species, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

        # Optionally save the annotated frame
        annotated_folder = 'output/annotated_frames'
        if not os.path.exists(annotated_folder):
            os.makedirs(annotated_folder)
        annotated_path = os.path.join(annotated_folder, os.path.basename(frame_file))
        cv2.imwrite(annotated_path, frame)

    # Step 4: Save counts to CSV
    save_counts_to_csv(species_counts, counts_csv)

    # Step 5: Visualize the results
    plot_species_counts(counts_csv)

if __name__ == '__main__':
    main()


Extracted 26 frames from /content/drive/MyDrive/Dev/BirdWatcher/Input/output.mp4


FileNotFoundError: [Errno 2] No such file or directory: 'yolov8_birds.pt'